In [249]:
# Imports

import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import datetime,os,warnings

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,Input,LSTM,Dropout,Conv2D,Conv2DTranspose,Activation,Add,Concatenate,BatchNormalization,Bidirectional

from sklearn.decomposition import PCA,FastICA
from sklearn.preprocessing import KBinsDiscretizer,OneHotEncoder,LabelEncoder
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import train_test_split

from scipy import optimize as opt

warnings.filterwarnings('ignore')
sns.set()

In [431]:
def err_metrics(x,y,s):
    err = np.linalg.norm(y-s,ord=2)
    energy = np.linalg.norm(x,ord=2) 
    pctg_active_TX = (x.shape[0]-(x==0).astype(int).sum())/x.shape[0]
    return err,energy,pctg_active_TX

In [135]:
# Parameters

num_RX = 5
num_TX = 100

blocklen = 1

L = 0.9
M = 0

In [403]:
# Assemble source message
s = rnd.randn(num_RX,blocklen)+1j*rnd.randn(num_RX,blocklen) 
#s = rnd.randn(num_RX,blocklen)

In [404]:
# Make channel
H = (np.matrix(rnd.randn(num_RX,num_TX))+1j*rnd.randn(num_RX,num_TX))/(num_TX*num_RX)
#H = (np.matrix(rnd.randn(num_RX,num_TX)))/(num_TX*num_RX)

In [405]:
# Use ZF Precoding
x_zf = (np.linalg.inv(H.H*H+L*np.ones(shape=[num_TX,num_TX]))*H.H)*s

In [432]:
# Transmit
y_zf = H*x_zf

# Calculate RX error, TX energy and TX active antennas 
err_zf,energy_zf,pctg_active_TX_zf = err_metrics(x_zf,y_zf,s)

In [441]:
def targetfun(x,LL=0.001):
    return np.linalg.norm(H*x.reshape(-1,1)-s,ord=2)+LL*np.linalg.norm(x,ord=2)

x0 = rnd.randn(num_TX,blocklen)+1j*rnd.randn(num_TX,blocklen)
res = opt.minimize(targetfun,x0)
print(res.message)
x_opt = res.x.reshape(-1,1)

Desired error not necessarily achieved due to precision loss.


In [442]:
# Transmit
y_opt = H*x_opt

# Calculate RX error, TX energy and TX active antennas 
err_opt,energy_opt,pctg_active_TX_opt = err_metrics(x_opt,y_opt,s)

In [443]:
err_opt

1.3159675178479615e-08

In [444]:
err_zf

0.004925718932793081